In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:

# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-31 23:21:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-31 23:21:01 (6.55 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("datafinal").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://reno8946-bucket.s3.us-east-2.amazonaws.com/test.csv"
spark.sparkContext.addFile(url)
test_df = spark.read.csv(SparkFiles.get("test.csv"), sep=",", header=True, inferSchema=True)

In [6]:
# Show DataFrame
test_df.show()

+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|_c0|    id|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+-

In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://reno8946-bucket.s3.us-east-2.amazonaws.com/train.csv"
spark.sparkContext.addFile(url)
train_df = spark.read.csv(SparkFiles.get("train.csv"), sep=",", header=True, inferSchema=True)

In [8]:
# Show DataFrame
train_df.show()

+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|_c0|    id|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+-

In [9]:
# Row count of train.df
train_df.count()

103904

In [10]:
#Concatenate dataframes
def append_dfs(train_df,test_df):
    list1 = train_df.columns
    list2 = test_df.columns
    for col in list2:
        if(col not in list1):
            train_df = train_df.withColumn(col, F.lit(None))
    for col in list1:
        if(col not in list2):
            test_df = test_df.withColumn(col, F.lit(None))
    return train_df.unionByName(test_df)
final_df = append_dfs(train_df,test_df)
final_df.show()

+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|_c0|    id|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+---+------+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+-

In [11]:
# row count of concatenated file
final_df.count()

129880

In [13]:
# Rename columns
final_df=final_df.withColumnRenamed("Food and drink", "Food_and_drink").withColumnRenamed("Type of Travel", "Type_of_Travel").withColumnRenamed("Departure/Arrival time convenient", "Departure/Arrival_time_convenient").withColumnRenamed("Ease of Online booking", "Ease_of_Online_booking").withColumnRenamed("Gate location", "Gate_location").withColumnRenamed("Departure Delay in Minutes", "Departure_Delay_in_Minutes").withColumnRenamed("Arrival Delay in Minutes", "Arrival_Delay_in_Minutes").withColumnRenamed("Customer Type", "Customer").withColumnRenamed("Inflight wifi service", "Inflight wifi").withColumnRenamed("Ease_of_Online_booking", "Ease_of_Online booking")


In [14]:
# Drop NAs
final_df=final_df.dropna()

In [15]:
# Drop index and id columns
airline_df=final_df.drop('_c0','id')
airline_df.show()

+------+-----------------+---+---------------+--------+---------------+-------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|Gender|         Customer|Age| Type_of_Travel|   Class|Flight Distance|Inflight wifi|Departure/Arrival_time_convenient|Ease_of_Online booking|Gate_location|Food_and_drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure_Delay_in_Minutes|Arrival_Delay_in_Minutes|        satisfaction|
+------+-----------------+---+---------------+--------+---------------+-------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+--------------

In [21]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://datafinal.c50h51d04cbn.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [22]:
# Write DataFrame to active_user table in RDS
airline_df.write.jdbc(url=jdbc_url, table='airline', mode=mode, properties=config)

In [23]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# follows django database settings format, replace with your own settings
DATABASES = {
    'datafinal':{
        'POSTGRES_DBNAME': 'postgres',
        'POSTGRES_USERNAME': 'postgres',
        'POSTGRES_PASSWORD': 'password',
        'POSTGRES_ADDRESS': 'datafinal.c50h51d04cbn.us-east-2.rds.amazonaws.com',
        'POSTGRES_PORT': 5432,
    },
}

# choose the database to use
db = DATABASES['datafinal']

# construct an engine connection string
engine_string =  ('postgresql+psycopg2://{username}:{password}@{ipaddress}:{port}/{dbname}'
               .format(username= db['POSTGRES_USERNAME'],
                      password= db['POSTGRES_PASSWORD'],
                      ipaddress= db['POSTGRES_ADDRESS'],
                      port= db['POSTGRES_PORT'],
                      dbname= db['POSTGRES_DBNAME'])
               )

# create sqlalchemy engine
engine = create_engine(engine_string)

# read a table from database into pandas dataframe, replace "tablename" with your table name
df = pd.read_sql_table('airline', engine)
df.head()

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


,Gender,Customer,Age,Type_of_Travel,Class,Flight Distance,Inflight wifi,Departure/Arrival_time_convenient,Ease_of_Online booking,Gate_location,Food_and_drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied


In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [28]:
# Create features
X = df.drop(columns="satisfaction", axis=1)
X = pd.get_dummies(X)

# Create target
y = df["satisfaction"]

X



,Age,Flight Distance,Inflight wifi,Departure/Arrival_time_convenient,Ease_of_Online booking,Gate_location,Food_and_drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes,Gender_Female,Gender_Male,Customer_Loyal Customer,Customer_disloyal Customer,Type_of_Travel_Business travel,Type_of_Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
0,13,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,0,1,1,0,0,1,0,0,1
1,25,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,0,1,0,1,1,0,1,0,0
2,26,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,1,0,1,0,1,0,1,0,0
3,25,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,1,0,1,0,1,0,1,0,0
4,61,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,0,1,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388456,39,3483,1,1,1,1,3,5,4,5,5,5,5,4,5,3,0,0.0,1,0,1,0,1,0,1,0,0
388457,42,3636,0,5,0,4,2,5,4,1,1,1,1,3,1,3,0,0.0,0,1,1,0,1,0,1,0,0
388458,64,376,1,5,1,1,2,4,5,4,4,1,4,3,4,4,0,0.0,1,0,1,0,0,1,0,1,0
388459,57,192,5,5,5,1,2,4,5,4,4,5,4,5,4,5,0,0.0,1,0,1,0,0,1,0,1,0


In [29]:
y.describe()

count                      388461
unique                          2
top       neutral or dissatisfied
freq                       219675
Name: satisfaction, dtype: object

In [30]:
#Encode text labels into numerical values
y_encoded = pd.get_dummies(y, columns=["satisfaction"])
y_encoded.head()

,neutral or dissatisfied,satisfied
0,1,0
1,1,0
2,0,1
3,1,0
4,0,1


In [31]:
# Check the balance of target values
y_encoded.value_counts()

neutral or dissatisfied  satisfied
1                        0            219675
0                        1            168786
dtype: int64

In [33]:
df.count()

Gender                               388461
Customer                             388461
Age                                  388461
Type_of_Travel                       388461
Class                                388461
Flight Distance                      388461
Inflight wifi                        388461
Departure/Arrival_time_convenient    388461
Ease_of_Online booking               388461
Gate_location                        388461
Food_and_drink                       388461
Online boarding                      388461
Seat comfort                         388461
Inflight entertainment               388461
On-board service                     388461
Leg room service                     388461
Baggage handling                     388461
Checkin service                      388461
Inflight service                     388461
Cleanliness                          388461
Departure_Delay_in_Minutes           388461
Arrival_Delay_in_Minutes             388461
satisfaction                    

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1,stratify=y)
Counter(y_train)

Counter({'neutral or dissatisfied': 1, 'satisfied': 1})

In [35]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(291345, 27)
(97116, 27)
(291345, 2)
(97116, 2)
